In [69]:
import numpy as np
import pandas as pd
import random

from sktime.classification.kernel_based import RocketClassifier
from tslearn.preprocessing import TimeSeriesResampler
from sklearn.metrics import accuracy_score

In [70]:
tank_training_data = pd.read_pickle("Train.pkl")

# simulates new data coming in
new_data = pd.read_csv("Batch_76.csv")


In [71]:
if new_data.iloc[0]["CuStepNo ValueY"] == 1 and tank_training_data.iloc[-1]["CuStepNo ValueY"] == 3:
    data_to_analyse = new_data
else:
    newest_batch = tank_training_data.iloc[-1]["Batch"]
    data_to_analyse = pd.concat([tank_training_data[tank_training_data["Batch"] == newest_batch], new_data])
    tank_training_data = tank_training_data[tank_training_data["Batch"] != newest_batch]

In [72]:
for i in tank_training_data["Batch"].unique():
    tank_training_data.drop(tank_training_data.loc[(tank_training_data["Batch"] == i)&(tank_training_data["CuStepNo ValueY"] == data_to_analyse["CuStepNo ValueY"].unique()[-1])][data_to_analyse[data_to_analyse["CuStepNo ValueY"] == data_to_analyse["CuStepNo ValueY"].unique()[-1]].__len__():].index,inplace=True)


In [73]:
X_train = [tank_training_data.loc[tank_training_data["Batch"]==i].drop(columns=["DeviationID ValueY", "Batch", "CuStepNo ValueY", "Next_Step", "timestamp", "Unnamed: 0"]).to_numpy() for i in tank_training_data["Batch"].unique()]
y_train = np.array([tank_training_data.loc[tank_training_data["Batch"]==i]["DeviationID ValueY"].unique()[0] for i in tank_training_data["Batch"].unique()])


In [76]:
data_amount_per_batch = [len(i) for i in X_train]
scaler = TimeSeriesResampler(sz=int(np.median(data_amount_per_batch)))
X_train = scaler.fit_transform(X_train)

In [77]:
X_train.shape

(251, 1050, 39)

In [78]:
X_train = X_train.reshape(X_train.shape[0],39,X_train.shape[1])

In [79]:
X_train.shape

(251, 39, 1050)

In [80]:
clf = RocketClassifier(num_kernels=500)
clf.fit(X_train, y_train)

RocketClassifier(num_kernels=500)

In [81]:
data_to_analyse.drop(columns=["DeviationID ValueY", "CuStepNo ValueY", "timestamp", "Unnamed: 0"], inplace=True)

In [82]:
data_to_analyse.to_numpy().reshape(39,data_to_analyse.__len__()).shape

(39, 1036)

In [83]:
scaler = TimeSeriesResampler(sz=int(np.median(data_amount_per_batch)))
test = scaler.fit_transform(data_to_analyse.to_numpy().reshape(1,data_to_analyse.__len__(),39))
clf.predict(test.reshape(test.shape[0],39,test.shape[1]))

In [85]:
test.reshape(test.shape[0],39,test.shape[1])

(1, 39, 1050)

In [28]:
data_to_analyse.shape

(1036, 1050, 1)

In [20]:
data_to_analyse.to_numpy().reshape(1,39,data_to_analyse.__len__()).shape

(1, 39, 1036)

In [21]:
clf.predict(data_to_analyse.to_numpy().reshape(1,39,data_to_analyse.__len__()))

array([9], dtype=int64)

In [66]:
X_train.shape

(251, 1050, 39)

In [65]:
test.shape

(1, 1050, 39)

In [68]:
test

array([[[1.56720276e+02, 1.00000000e+02, 9.54861641e-02, ...,
         2.00000003e-01, 0.00000000e+00, 1.73006020e-03],
        [1.56763243e+02, 1.00000000e+02, 9.60570834e-02, ...,
         2.00000003e-01, 0.00000000e+00, 1.70322646e-03],
        [1.56807755e+02, 1.00000000e+02, 9.49382965e-02, ...,
         2.00000003e-01, 0.00000000e+00, 1.71129768e-03],
        ...,
        [1.51209031e+02, 1.00000000e+02, 8.62577144e-02, ...,
         2.00000003e-01, 0.00000000e+00, 1.73268749e-03],
        [1.51405808e+02, 1.00000000e+02, 8.73919481e-02, ...,
         2.00000003e-01, 0.00000000e+00, 1.74134004e-03],
        [1.51513000e+02, 1.00000000e+02, 8.79629850e-02, ...,
         2.00000003e-01, 0.00000000e+00, 1.72441127e-03]]])

In [86]:
clf.predict(test.reshape(test.shape[0],39,test.shape[1]))

array([3], dtype=int64)